In [1]:
from topic_modelling.BERTopicModel import BERTopicModel
from ontology_generation.OntologyGen import OntologyGen
from ontology_generation.OntologyEncap import OntologyEncap
from ontology_generation.Evaluation import Evaluation
import re
import pandas as pd
from sentence_transformers import SentenceTransformer
import os
from rdflib import Namespace, Graph

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\shrut\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\shrut\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
def validate_folder_path(folder_path):
    """
    Validate if the folder exists and contains CSV files.

    Parameters:
    - folder_path (str): Path to the folder.

    Returns:
    - bool: True if the folder exists and contains CSV files, False otherwise.
    """
    if not os.path.isdir(folder_path):
        print(f"Error: Folder '{folder_path}' does not exist.")
        return False
    elif not any(filename.endswith('.csv') for filename in os.listdir(folder_path)):
        print(f"Warning: Folder '{folder_path}' does not contain any CSV files.")
        return False
    else:
        return True
    
def validate_input_data(df):
    """
    Validate input DataFrame.

    Parameters:
    - df (pandas.DataFrame): Input DataFrame.

    Returns:
    - bool: True if input data is valid, False otherwise.
    """
    # Check if DataFrame has required columns
    required_columns = ['abstract', 'type', 'language', 'title']
    if not all(col in df.columns for col in required_columns):
        print("Error: DataFrame is missing required columns.")
        return False
    
    # Perform additional validation as needed

    return True

def run_model(folder_path, topic_name, messages, subsample_percentage):
    try:
        if os.path.exists(folder_path):
            if validate_folder_path(folder_path):
                # Select the CSV file containing the word "cleaned" in its filename
                csv_files = [filename for filename in os.listdir(folder_path) if filename.endswith('.csv') and "cleaned" in filename]
                if csv_files:
                    # If there are files containing "cleaned", select the first one
                    csv_file = csv_files[0]
                    file_path = os.path.join(folder_path, csv_file)
                else:
                    raise ValueError("Data not found")
            # else:
            #     print("Folder does not contain any CSV files. Running openalex.get_data...")
            #     # Run openalex.get_data
            #     openalex = OpenAlex()
            #     openalex.get_data("natural language processing", "topics", "data")
            #     # Attempt to load the CSV file again
            #     csv_files = [filename for filename in os.listdir(folder_path) if filename.endswith('.csv') and "cleaned" in filename]
            #     if not csv_files:
            #         raise ValueError("No CSV file containing 'cleaned' in its filename found.")
        else:
            raise ValueError("Folder path does not exist.")

        if csv_files:
            # Load the selected CSV file
            df = pd.read_csv(file_path)
            # Perform further processing with df
            # Clean the abstract column in the dataframe
            if validate_input_data(df):

                # Filter the dataframe based on certain conditions
                subset = df[(df['abstract'].str.len() > 10) & 
                        (df['type'] == 'article') & 
                        (df['language'] == 'en') & 
                        (df['title'].str.len() > 10)]

                # Calculate the number of rows for the subsample
                subsample_size = int(len(subset) * (subsample_percentage / 100))

                # Randomly sample the data
                subsample = subset.sample(n=subsample_size, random_state=42)

                # Split the title and abstract documents into lists
                title_docs = subsample["title"].to_list()
                abstract_docs = subsample["abstract"].to_list()

                # Model
                sentence_model = SentenceTransformer('all-mpnet-base-v2', device="cuda")
                embeddings = sentence_model.encode(title_docs, show_progress_bar=True)
                bert_base_model = BERTopicModel(embeddings=embeddings, reduce_outliers=True, messages=messages)
                results, freq, topic_dict, topics, topic_model = bert_base_model.train_model(title_docs)

                # Add the Topic and Topic Words to the subsample dataframe
                subsample["Topic"] = topics
                topic_dict_words = {x: [i[0] for i in topic_dict[x]] for x in topic_dict}
                topic_dict_first_word = {x: topic_dict[x][0][0] for x in topic_dict}
                subsample['topic_words'] = subsample['Topic'].map(topic_dict_words)
                subsample['topic_first_word'] = subsample['Topic'].map(topic_dict_first_word)

                subsample.to_csv("../model/subsample.csv")

                embedding_model = "sentence-transformers/all-MiniLM-L6-v2"
                topic_model.save("../model/", serialization="safetensors", save_ctfidf=True, save_embedding_model=embedding_model)

                return subsample
    except Exception as e:
        print(f"An error occurred: {e}")

In [3]:
folder_path = "data/"
topic_name = "natural language processing"
excluded_words = "natural language processing, computer science, machine learning, artificial intelligence"
messages = [
            {"role": "system", "content": f"You are a topic representation creator model for studies in the domain of {topic_name}. Your task is to determine the sub-domain of the research work based on its abstract. Each sub-domain name should not exceed more than 4 words. Your representations should be specific and focus on the most described object. Always provide a representation. Please do not use the following words delimited with triple backticks: '''natural language processing, computer science, machine learning, artificial intelligence'''"},
            {'role': 'user', 'content': """I have a topic that contains the following documents which are delimited with triple backticks:
        '''-  business world large companies that can achieve continuity in innovation gain a significant competitive advantage the sensitivity of these companies to follow and monitor news sources in e commerce social media and forums provides important information to businesses in decision making process large amount of data shared in these resources sentiment analysis can be made from people s comments about services and products users emotions can be extracted and important feedback can be obtained all of this is of course possible with accurate sentiment analysis this study new data sets were created for turkish english and arabic and for first time comparative sentiment analysis was performed from texts in three different languages addition a very comprehensive study was presented to researchers by comparing performances of both pre trained language models for turkish arabic and english as well as deep learning and machine learning models our paper will guide researchers working on sentiment analysis about which methods will be more successful in texts written in different languages which contain different types and spelling mistakes which factors will affect success and how much these factors will affect performance,
        - analysis also called opinion mining is field of study that analyzes people s opinions sentiments attitudes and emotions are important sentiment analysis since songs and mood are mutually dependent on each other on selected song it becomes easy find mood of listener future it can be used for recommendation the song lyric is a rich source of datasets containing words that are helpful analysis and classification of sentiments generated from it now a days observe a lot of inter sentential and intra sentential code mixing songs which has a varying impact on audience to study this impact created a telugu songs dataset which contained both telugu english code mixed and pure telugu songs in this paper classify songs based on its arousal as exciting or non exciting we develop a language identification tool and introduce code mixing features obtained from it as additional features system with these additional features attains 4 5 accuracy greater than traditional approaches on our dataset, 
        - this paper we propose a sentiment analysis model for the assessment of teacher performance in the classroom by tweets written by a pilot group of college students naive bayes nb is the technique to be applied to classify tweets based on the polar express emotion positive negative and neutral to carry out this process a dataset fits adding distinctive terms of context as possible features to support the classification process, 
        - analysis refers to classify emotion of a text whether positive or negative the studies conducted on sentiment analysis are generally based on english and other languages while there are limited studies on turkish in this study after constructing a dataset using a well known hotel reservation site booking com compare performances of different machine learning approaches we also apply dictionary based method sentitfidf which differs from traditional methods due to their logarithmic differential term frequency and term presence distribution usage the results are evaluated using area under of a receiver operating characteristic roc curve auc the results show that using document term matrix as input gives better classification results than tfidf matrix we also observe that best results are obtained using random forest classifier with an auc value of 89 on both positive and negative comments, 
        - the current era of computing the use of social networking sites like twitter and facebook is growing significantly over time people from different cultures and backgrounds share vast volumes of textual comments that show their viewpoints on several aspects of life and make them available to all for commenting monitoring real social media activities has now become a prime concern for politicians in understanding their social image this paper are going to analyse the tweets of various social media platforms regarding two prominent political leaders and classify them as positive negative or neutral using machine learning and deep learning methods we have proposed a deep learning approach for a better solution our proposed model has provided state of the art results using deep learning models'''
        It must be in the following format: <topic label>"""},
            {'role': 'assistant', 'content': 'Sentiment Analysis'},
            {"role": "user", "content": f"""I have a topic that contains the following documents which are delimited with triple backticks:
        '''[DOCUMENTS]'''
        REMEMBER to only use 1-4 words and to NOT use the following words delimited with triple backticks: '''{excluded_words}'''
        It must be in the following format: <topic label>"""}
]

In [4]:
# print("Running model...")
# df = run_model("data/", topic_name, messages, 5)
# print("Model run completed.")

In [5]:
df = pd.read_csv("data/nlp_subsample_withTopics.csv")

ontogen = OntologyGen(model_name="gpt-4-1106-preview", deployment_name="gpt_chat_test_preview")
taxonomy_topics_df = ontogen.extract_concepts(df, topic_name, add_OAconcepts = False)

Number of concepts before LLM refinement:  72
Number of concepts after LLM refinement:  41


In [6]:
# Taxonomy creation
role = f"You are an ontology engineer, tasked with helping build an ontology for technology monitoring in the domain of {topic_name}"
eg = '''Create a taxonomy for NLP based on the following topics:
- Question Answering
- Semantic Similarity
- Chatbot
- Annotation
- Co-occurrence
Desired format: {
"Category 1": ["topic 1", "topic 5","..."],
"Category 2": [{"topic 3" : ["topic 6", "topic 8"]},"topic 9","topic 11", "..."],
"Category 3": ["topic 2", "topic 4","..."]
}'''
answer = '''{
     "Semantic Text Processing": [{"Semantic Similarity" : ["Concept Similarity", "Semantic Distance", "Sentence Similarity", "Word Similarity"]}],
     "Natural Language Interfaces": ["Question Answering", "Chatbot"],
     "Text Processing: ["Annotation", "Co-occurrence"]
}'''

taxonomy = ontogen.taxonomy_creation(role, taxonomy_topics_df, topics_per_batch = 20, eg=eg, answer=answer)

In [7]:
# Determining relations

prompt = "Given the following taxonomy:\n" + str(taxonomy) + '''
\n\Modify the taxonomy with the relations between the key and values with superTopicof, contributesTo. Use relation relatedEquivalent only if two topics are same'''

prompt += '''Return the taxonomy only in the following format. Check that all parenthesis are closed:
    {
     "Category 1": { "relation 1": ["topic 1", "topic 5","..."]},
     "Category 2": { "relation 1" : [{"topic 3": { "relation 2": ["topic 6", "topic 8"]}}, "topic 9","topic 11", "..."]},
}''' + '''\nDO NOT ADD FILLER WORDS LIKE "NEWLY ADDED"'''

eg = '''Given the following taxonomy: 
{
     "Semantic Text Processing": [{"Semantic Similarity" : ["Concept Similarity", "Semantic Distance", "Sentence Similarity", "Word Similarity"]}]},
     "Natural Language Interfaces": ["Question Answering", "Chatbot"],
     "Text Processing: ["Annotation", "Co-occurrence"]
}
Define the relations between the key and values with superTopicof, contributesTo. Use relation relatedEquivalent only if two topics are same.\n Desired format:
    {
     "Category 1": { "relation 1": ["topic 1", "topic 5","..."]},
     "Category 2": { "relation 1" : [{"topic 3": { "relation 2": ["topic 6", "topic 8"]}}, "topic 9","topic 11", "..."]},
     "Category 3": {"relation 2": ["topic 13"], "relation 1": ["topic 14", "topic 17"]}
}'''

# AlternativeLabelOf, IsParentOf, Includes, IsRelatedTo.
answer = '''{
  "Semantic Text Processing": {
    "IsParentOf": [
      {
        "Semantic Similarity": {
          "Includes": [
            "Concept Similarity",
            "Semantic Distance",
            "Sentence Similarity",
            "Word Similarity"
          ]
        }
      }
    ]
  },
  "Natural Language Interfaces": {
    "contributesTo": [
      "Question Answering",
      "Chatbot"
    ]
  },
  "Text Processing": {
    "superTopicof": [
      "Annotation",
      "Co-occurrence"
    ]
  }
}'''

print("Adding relations")
taxonomy = ontogen.prompt_extract(role, prompt)
print("Relation prompting completed.")

Adding relations
Relation prompting completed.


In [8]:
taxonomy

{'Text Analysis and Classification': {'superTopicOf': ['Amazon Reviews',
   'Text Classification',
   'Named Entity Recognition',
   'Clinical Text Analysis',
   "Students' Feedback Analysis",
   'Tourist Comment Classification',
   'Arabic Language Analysis',
   'Text Similarity Studies',
   'Text Summarization',
   'Text Analysis'],
  'contributesTo': [{'Opinion Mining Approach': {'superTopicOf': ['Logic Rules in Sentiment Classification',
      'Fuzzy Quantification in Hotel Reviews']}},
   {'Sentiment Analysis': {'relatedEquivalent': ['Sentiment Analysis Applications'],
     'superTopicOf': ['Online Pharmacy Reviews',
      'Twitter Sentiment Analysis',
      'Online Reviews Sentiment Analysis',
      'News Reviews Sentiment Lexicon',
      'Twitter Sentiments Analysis']}}]},
 'Machine Learning Models and Approaches': {'superTopicOf': ['Feature Selection',
   'Neutrosophic Sets',
   'Majority Voting Approach',
   'Fuzzy Integral Classifier Fusion',
   'Word Embeddings'],
  'contrib

In [9]:
# Update taxonomy

prompt = f'''You have a taxonomy for {topic_name} concepts. Your task is to:

1. Reorganize the taxonomy by introducing more levels and sub-hierarchies to better group related concepts.
2. Review the 'Irrelevant' category and identify any topics that are actually relevant to {topic_name} or related fields.
3. For relevant topics from 'Irrelevant', decide where to place them within the reorganized taxonomy structure (under existing categories, new subcategories, or new top-level categories if needed).
4. For remaining irrelevant topics, delete them from the {topic_name} taxonomy and introduce separate top-level dictionaries to categorize them into and ensure that the dictionaries don't contain themselves.

Original taxonomy:\n''' + str(taxonomy) + '''\nReturn ONLY the modified taxonomy and ensure that there exists relations between structures. NO captions'''

taxonomy = ontogen.prompt_extract(role, prompt)
print("Taxonomy update completed.")

Taxonomy update completed.


In [10]:
taxonomy

{'Natural Language Processing Fundamentals': {'superTopicOf': ['Text Analysis and Classification',
   'Machine Learning Models and Approaches',
   'Natural Language Understanding and Generation',
   'Specialized Text and Speech Processing'],
  'contributesTo': [{'Natural Language Processing Applications': {'superTopicOf': ['Sentiment Analysis',
      'Opinion Mining Approach']}}]},
 'Text Analysis and Classification': {'superTopicOf': ['Amazon Reviews',
   'Text Classification',
   'Named Entity Recognition',
   'Clinical Text Analysis',
   "Students' Feedback Analysis",
   'Tourist Comment Classification',
   'Arabic Language Analysis',
   'Text Similarity Studies',
   'Text Summarization',
   'Text Analysis'],
  'contributesTo': [{'Sentiment Analysis': {'relatedEquivalent': ['Sentiment Analysis Applications'],
     'superTopicOf': ['Online Pharmacy Reviews',
      'Twitter Sentiment Analysis',
      'Online Reviews Sentiment Analysis',
      'News Reviews Sentiment Lexicon']}}]},
 'M

In [11]:
# Manual organization
print("Reorganizing taxonomy...")
categories = [topic_name.capitalize()]

# Example 
# main_categories = [
#     'Natural Language Processing',
#     'Artificial Intelligence',
#     'Computer Science'
# ]


taxonomy = ontogen.reorganize_taxonomy(taxonomy, categories)
print("Taxonomy reorganization completed.")

Reorganizing taxonomy...
Taxonomy reorganization completed.


In [12]:
taxonomy

{'Natural language processing': {'superTopicOf': {'Natural Language Processing Fundamentals': {'superTopicOf': ['Text Analysis and Classification',
     'Machine Learning Models and Approaches',
     'Natural Language Understanding and Generation',
     'Specialized Text and Speech Processing'],
    'contributesTo': [{'Natural Language Processing Applications': {'superTopicOf': ['Sentiment Analysis',
        'Opinion Mining Approach']}}]},
   'Text Analysis and Classification': {'superTopicOf': ['Amazon Reviews',
     'Text Classification',
     'Named Entity Recognition',
     'Clinical Text Analysis',
     "Students' Feedback Analysis",
     'Tourist Comment Classification',
     'Arabic Language Analysis',
     'Text Similarity Studies',
     'Text Summarization',
     'Text Analysis'],
    'contributesTo': [{'Sentiment Analysis': {'relatedEquivalent': ['Sentiment Analysis Applications'],
       'superTopicOf': ['Online Pharmacy Reviews',
        'Twitter Sentiment Analysis',
      

In [13]:
# Export taxonomy as a json file - Prompt is hard coded
ontogen.taxonomy_json(role, taxonomy)
print("Taxonomy exported as JSON.")

Taxonomy exported as JSON.


In [14]:
# Extract topics from taxonomy for term typing
terms = ontogen.extract_topics(taxonomy)
role = "You are a topic summarizer model"
temp = f'''You are a summarizer model tasked with writing one line descriptions for ontology creation for the {topic_name} domain. 
Describe the following topics in one line: ''' + ', '.join(terms) + '''
Return the response only in the dictionary format. USE proper QUOTES:
{
'topic1': 'description1',
'topic2': 'description2',
}
'''
print("Prompting for topic descriptions...")
topics_descriptions = ontogen.prompt_extract(role, temp)
print("Term typing completed")

Prompting for topic descriptions...
Term typing completed


In [15]:
topics_descriptions

{'Image Captioning': 'Generating textual description of images using AI models.',
 'Amazon Reviews': 'Analyzing consumer feedback on products listed on Amazon.',
 'Opinion Mining Approach': 'Techniques for extracting subjective information from text.',
 'Feature Selection': 'Identifying the most relevant features for model training.',
 'Text Classification': 'Automatically categorizing text into predefined groups.',
 'Neutrosophic Sets': 'Mathematical tool for dealing with imprecise, indeterminate data.',
 'Parsing Strategies': 'Methods for analyzing and understanding grammatical structure of sentences.',
 'Handwritten Word Recognition': 'Techniques for converting handwritten text into digital format.',
 'Sentiment Analysis Applications': 'Utilizing sentiment analysis in various domains like marketing and customer service.',
 'Logic Rules in Sentiment Classification': 'Using predefined logical rules to improve sentiment analysis accuracy.',
 'Fuzzy Quantification in Hotel Reviews': 'Ap

In [16]:
# Define namespaces
fh= Namespace('http://fraunhofer.de/example/')  # Namespace for our entities
schema = Namespace("http://schema.org/")  # Common vocabulary for attributes
wiki= Namespace('https://www.wikidata.org/wiki/')

ontoencap = OntologyEncap(fh, schema, wiki)
g = ontoencap.initiate_graph()

In [17]:
try:
    # Print or serialize RDF triples (turtle format)
    print("Processing data for ontology encoding")
    graph = ontoencap.process_data(taxonomy, topics_descriptions)
    turtle_data = graph.serialize(format="turtle")
    print(turtle_data)
except Exception as e:
    print("Error occurred during serialization:", e)

# Assuming 'data' is the provided data dictionary
graph.serialize(destination=f'output/taxonomy.ttl' , format="turtle")

http://schema.org/Natural Language Processing Fundamentals does not look like a valid URI, trying to serialize this will break.
http://schema.org/Text Analysis and Classification does not look like a valid URI, trying to serialize this will break.
http://schema.org/Machine Learning Models and Approaches does not look like a valid URI, trying to serialize this will break.
http://schema.org/Natural Language Understanding and Generation does not look like a valid URI, trying to serialize this will break.
http://schema.org/Specialized Text and Speech Processing does not look like a valid URI, trying to serialize this will break.
http://schema.org/Natural Language Processing Applications does not look like a valid URI, trying to serialize this will break.


Processing data for ontology encoding
@prefix ns1: <http://schema.org/> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .

<http://fraunhofer.de/example/Amazon_Reviews> a rdfs:Class ;
    rdfs:label "Amazon Reviews" ;
    rdfs:comment "Analyzing consumer feedback on products listed on Amazon." .

<http://fraunhofer.de/example/Arabic_Language_Analysis> a rdfs:Class ;
    rdfs:label "Arabic Language Analysis" ;
    rdfs:comment "Techniques and challenges in processing Arabic text and speech." .

<http://fraunhofer.de/example/Clinical_Text_Analysis> a rdfs:Class ;
    rdfs:label "Clinical Text Analysis" ;
    rdfs:comment "Extracting meaningful information from medical records and clinical notes." .

<http://fraunhofer.de/example/Dialog_Systems> a rdfs:Class ;
    rdfs:label "Dialog Systems" ;
    rdfs:comment "AI systems capable of conducting conversation with human users." .

<http://fraunhofer.de/example/Ensemble-Hybrid_Model> a rdfs:Class ;
    rdfs:label "Ensemble-Hybrid Mode

<Graph identifier=N18bf125f4c8746e0a8ca8df37ca4f62e (<class 'rdflib.graph.Graph'>)>